In [0]:
import os, numpy as np, pandas as pd, cv2
from keras.layers import Dropout, Dense
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model
from keras.applications import VGG16

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=xxxxxxxxxxxx-xxxxxxxxxxxxxxxxxxxxxi.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp /content/drive/My\ Drive/animated_images_final.zip /content/animated_images_final.zip
!cp /content/drive/My\ Drive/animated_metadata_final.pkl /content/animated_metadata_final.pkl
!cp /content/drive/My\ Drive/human_images.zip /content/human_images.zip
!cp /content/drive/My\ Drive/human_metadata_final.pkl /content/human_metadata_final.pkl

In [0]:
!unzip animated_images_final.zip -d animated_images_final
!unzip human_images.zip -d human_images

Archive:  animated_images_final.zip
   creating: animated_images_final/animated_images_final/
  inflating: animated_images_final/animated_images_final/ray_anger_116.png  
  inflating: animated_images_final/animated_images_final/malcolm_joy_382.png  
  inflating: animated_images_final/animated_images_final/bonnie_surprise_147.png  
  inflating: animated_images_final/animated_images_final/bonnie_anger_1486.png  
  inflating: animated_images_final/animated_images_final/ray_joy_83.png  
  inflating: animated_images_final/animated_images_final/bonnie_anger_1601.png  
  inflating: animated_images_final/animated_images_final/malcolm_sadness_699.png  
  inflating: animated_images_final/animated_images_final/aia_sadness_282.png  
  inflating: animated_images_final/animated_images_final/malcolm_surprise_189.png  
  inflating: animated_images_final/animated_images_final/jules_anger_1300.png  
  inflating: animated_images_final/animated_images_final/malcolm_sadness_497.png  
  inflating: animated_

In [0]:
animated_metadata_final = pd.read_pickle("animated_metadata_final.pkl")
animated_metadata_final['user.id'] = "animated_images_final/"
animated_metadata_final.head()

,user.id,image,emotion,Labels
0,animated_images_final/,mery_surprise_240.png,surprise,7
1,animated_images_final/,ray_surprise_1092.png,surprise,7
2,animated_images_final/,jules_fear_355.png,fear,3
3,animated_images_final/,jules_sadness_907.png,sadness,6
4,animated_images_final/,jules_sadness_715.png,sadness,6


In [0]:
human_metadata_final = pd.read_pickle("human_metadata_final.pkl")
human_metadata_final['user.id'] = "human_images/"
human_metadata_final.head()

,user.id,image,emotion,Labels
0,human_images/,Dale_Earnhardt_Jr_0001.jpg,happiness,4
1,human_images/,Mark_Philippoussis_0001.jpg,neutral,5
2,human_images/,Dennis_Kucinich_0006.jpg,anger,1
3,human_images/,Monica_Seles_0003.jpg,anger,1
4,human_images/,Luiz_Inacio_Lula_da_Silva_0005.jpg,neutral,5


In [0]:
frames = [animated_metadata_final, human_metadata_final]
combined_train = pd.concat(frames)
combined_train.shape

(10131, 4)

In [0]:
combined_train.reset_index(inplace = True, drop = True)
combined_train = combined_train.sample(frac = 1.0)
combined_train.reset_index(inplace = True, drop = True)
combined_train.head()

,user.id,image,emotion,Labels
0,animated_images_final/,jules_surprise_612.png,surprise,7
1,animated_images_final/,mery_surprise_671.png,surprise,7
2,animated_images_final/,aia_disgust_1312.png,disgust,2
3,animated_images_final/,bonnie_fear_991.png,fear,3
4,animated_images_final/,jules_surprise_531.png,surprise,7


In [0]:
combined_train_batch_pointer = 0
combined_labels = pd.get_dummies(combined_train['Labels']).as_matrix()
combined_labels.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


(10131, 7)

In [0]:
def loadBatch(batch_size):
  global combined_train_batch_pointer
  batch_images = []
  batch_labels = []
  for i in range(batch_size):
    folder_name = combined_train.iloc[combined_train_batch_pointer + i]['user.id']
    image_name = combined_train.iloc[combined_train_batch_pointer + i]['image']
    image = cv2.imread(folder_name+folder_name+image_name)
    cv2.imwrite(folder_name+folder_name+image_name, cv2.resize(image, (350, 350)))
    image_read = cv2.imread(folder_name+folder_name+image_name)
    image_final = image_read/255.0
    print(image_final.shape, end=" ")
    batch_images.append(image_final)
    batch_labels.append(combined_labels[combined_train_batch_pointer + i])
  print()
  combined_train_batch_pointer += batch_size
  return np.array(batch_images), np.array(batch_labels)

In [0]:
!rm -r bottleneck_combined_train
!rm -r combined_train_labels

rm: cannot remove 'bottleneck_combined_train': No such file or directory
rm: cannot remove 'combined_train_labels': No such file or directory


In [0]:
!mkdir bottleneck_combined_train
!mkdir combined_train_labels

In [0]:
model = VGG16(weights='imagenet', include_top=False)
batch_size = 10
for i in range(int(len(combined_train)/batch_size)):
  x, y = loadBatch(batch_size)
  print("Batch {} loaded".format(i+1))
  print (y.shape)
  np.save("combined_train_labels/"+"bottleneck_labels_{}".format(i+1),y)
  print("Creating bottleneck features for batch {}". format(i+1))
  print (x.shape)
  bottleneck_features = model.predict(x)
  np.save("bottleneck_combined_train/"+"bottleneck_{}".format(i+1), bottleneck_features)
  print("Bottleneck features for batch {} created and saved\n".format(i+1))





58892288/58889256 [==============================] - 1s 0us/step


(350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) 
Batch 1 loaded
(10, 7)
Creating bottleneck features for batch 1
(10, 350, 350, 3)
Bottleneck features for batch 1 created and saved

(350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) 
Batch 2 loaded
(10, 7)
Creating bottleneck features for batch 2
(10, 350, 350, 3)
Bottleneck features for batch 2 created and saved

(350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) 
Batch 3 loaded
(10, 7)
Creating bottleneck features for batch 3
(10, 350, 350, 3)
Bottleneck features for batch 3 created and saved

(350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350, 3) (350, 350

In [0]:
!zip -r bottleneck_combined_train.zip bottleneck_combined_train
!zip -r combined_train_labels.zip combined_train_labels

  adding: bottleneck_combined_train/ (stored 0%)
  adding: bottleneck_combined_train/bottleneck_68.npy (deflated 67%)
  adding: bottleneck_combined_train/bottleneck_398.npy (deflated 65%)
  adding: bottleneck_combined_train/bottleneck_743.npy (deflated 67%)
  adding: bottleneck_combined_train/bottleneck_480.npy (deflated 66%)
  adding: bottleneck_combined_train/bottleneck_583.npy (deflated 66%)
  adding: bottleneck_combined_train/bottleneck_929.npy (deflated 66%)
  adding: bottleneck_combined_train/bottleneck_378.npy (deflated 66%)
  adding: bottleneck_combined_train/bottleneck_794.npy (deflated 66%)
  adding: bottleneck_combined_train/bottleneck_575.npy (deflated 66%)
  adding: bottleneck_combined_train/bottleneck_75.npy (deflated 66%)
  adding: bottleneck_combined_train/bottleneck_211.npy (deflated 65%)
  adding: bottleneck_combined_train/bottleneck_344.npy (deflated 66%)
  adding: bottleneck_combined_train/bottleneck_699.npy (deflated 65%)
  adding: bottleneck_combined_train/bottlen

In [0]:
!cp /content/bottleneck_combined_train.zip /content/drive/My\ Drive/bottleneck_combined_train.zip
!cp /content/combined_train_labels.zip /content/drive/My\ Drive/combined_train_labels.zip

cp: cannot create regular file '/content/drive/My Drive/bottleneck_combined_train.zip': Input/output error


In [0]:
!cp /content/drive/My\ Drive/bottleneck_combined_train.zip /content/bottleneck_combined_train.zip
!cp /content/drive/My\ Drive/combined_train_labels.zip /content/combined_train_labels.zip

In [0]:
!unzip bottleneck_combined_train.zip -d bottleneck_combined_train
!unzip combined_train_labels.zip -d combined_train_labels

Archive:  bottleneck_combined_train.zip
   creating: bottleneck_combined_train/bottleneck_combined_train/
  inflating: bottleneck_combined_train/bottleneck_combined_train/bottleneck_851.npy  
  inflating: bottleneck_combined_train/bottleneck_combined_train/bottleneck_955.npy  
  inflating: bottleneck_combined_train/bottleneck_combined_train/bottleneck_546.npy  
  inflating: bottleneck_combined_train/bottleneck_combined_train/bottleneck_11.npy  
  inflating: bottleneck_combined_train/bottleneck_combined_train/bottleneck_823.npy  
  inflating: bottleneck_combined_train/bottleneck_combined_train/bottleneck_492.npy  
  inflating: bottleneck_combined_train/bottleneck_combined_train/bottleneck_53.npy  
  inflating: bottleneck_combined_train/bottleneck_combined_train/bottleneck_862.npy  
  inflating: bottleneck_combined_train/bottleneck_combined_train/bottleneck_825.npy  
  inflating: bottleneck_combined_train/bottleneck_combined_train/bottleneck_436.npy  
  inflating: bottleneck_combined_tra

In [0]:
no_of_classes = 7

In [0]:
def model(input_shape):
  model = Sequential()
  model.add(Dense(512, activation='relu', input_dim=input_shape))
  model.add(Dropout(0.1))
  model.add(Dense(256, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(64, activation='relu'))
  model.add(Dense(output_dim = no_of_classes, activation='softmax'))
  return model

In [0]:
input_shape = 10*10*512
model = model(input_shape)
model.summary()
model.compile(loss = 'categorical_crossentropy', optimizer="adam", metrics=["accuracy"])
epochs = 20
batch_size = 10
step = 0
bottleneck_combined_train_files = int(len(combined_train)/batch_size)
for epoch in range(epochs):
  avg_epoch_train_loss, avg_epoch_train_accuracy = 0, 0
  for i in range(bottleneck_combined_train_files):
    step += 1
    x_load = np.load("bottleneck_combined_train/"+"bottleneck_{}.npy".format(i+1))
    x = x_load.reshape(x_load.shape[0], x_load.shape[1]*x_load.shape[2]*x_load.shape[3])
    y = np.load("combined_train_labels/"+"bottleneck_labels_{}.npy".format(i+1))
    loss, accuracy = model.train_on_batch(x,y)
    print (" Step: {}".format(step))
    avg_epoch_train_loss += loss / bottleneck_combined_train_files
    avg_epoch_train_accuracy += accuracy / bottleneck_combined_train_files
  print(" Epoch: {}, Loss: {}, Accuracy: {}".format(epoch+1, avg_epoch_train_loss, avg_epoch_train_accuracy))
    
  

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               26214912  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_____

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=7)`
  if __name__ == '__main__':


 Step: 1
 Step: 2
 Step: 3
 Step: 4
 Step: 5
 Step: 6
 Step: 7
 Step: 8
 Step: 9
 Step: 10
 Step: 11
 Step: 12
 Step: 13
 Step: 14
 Step: 15
 Step: 16
 Step: 17
 Step: 18
 Step: 19
 Step: 20
 Step: 21
 Step: 22
 Step: 23
 Step: 24
 Step: 25
 Step: 26
 Step: 27
 Step: 28
 Step: 29
 Step: 30
 Step: 31
 Step: 32
 Step: 33
 Step: 34
 Step: 35
 Step: 36
 Step: 37
 Step: 38
 Step: 39
 Step: 40
 Step: 41
 Step: 42
 Step: 43
 Step: 44
 Step: 45
 Step: 46
 Step: 47
 Step: 48
 Step: 49
 Step: 50
 Step: 51
 Step: 52
 Step: 53
 Step: 54
 Step: 55
 Step: 56
 Step: 57
 Step: 58
 Step: 59
 Step: 60
 Step: 61
 Step: 62
 Step: 63
 Step: 64
 Step: 65
 Step: 66
 Step: 67
 Step: 68
 Step: 69
 Step: 70
 Step: 71
 Step: 72
 Step: 73
 Step: 74
 Step: 75
 Step: 76
 Step: 77
 Step: 78
 Step: 79
 Step: 80
 Step: 81
 Step: 82
 Step: 83
 Step: 84
 Step: 85
 Step: 86
 Step: 87
 Step: 88
 Step: 89
 Step: 90
 Step: 91
 Step: 92
 Step: 93
 Step: 94
 Step: 95
 Step: 96
 Step: 97
 Step: 98
 Step: 99
 Step: 100
 Step: 1

In [0]:
model.save("model.h5")

In [0]:
!cp /content/model.h5 /content/drive/My\ Drive/model.h5

In [0]:
model.save_weights("model_weights.h5")

In [0]:
!cp /content/model_weights.h5 /content/drive/My\ Drive/model_weights.h5